In [17]:
import pandas as pd

In [18]:
df = pd.read_csv("../prime/raw/kg.csv")

/var/folders/7r/2xd4t0f94wd49mswwc461j940000gn/T/ipykernel_1785/413405608.py:1: DtypeWarning: Columns (3,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../prime/raw/kg.csv")


In [19]:
df

,relation,display_relation,x_index,x_id,x_type,x_name,x_source,y_index,y_id,y_type,y_name,y_source
0,protein_protein,ppi,0,9796,gene/protein,PHYHIP,NCBI,8889,56992,gene/protein,KIF15,NCBI
1,protein_protein,ppi,1,7918,gene/protein,GPANK1,NCBI,2798,9240,gene/protein,PNMA1,NCBI
2,protein_protein,ppi,2,8233,gene/protein,ZRSR2,NCBI,5646,23548,gene/protein,TTC33,NCBI
3,protein_protein,ppi,3,4899,gene/protein,NRF1,NCBI,11592,11253,gene/protein,MAN1B1,NCBI
4,protein_protein,ppi,4,5297,gene/protein,PI4KA,NCBI,2122,8601,gene/protein,RGS20,NCBI
...,...,...,...,...,...,...,...,...,...,...,...,...
8100493,anatomy_protein_absent,expression absent,66747,4720,anatomy,cerebellar vermis,UBERON,5259,140,gene/protein,ADORA3,NCBI
8100494,anatomy_protein_absent,expression absent,63824,1377,anatomy,quadriceps femoris,UBERON,58254,105378952,gene/protein,KLF18,NCBI
8100495,anatomy_protein_absent,expression absent,63826,1379,anatomy,vastus lateralis,UBERON,58254,105378952,gene/protein,KLF18,NCBI
8100496,anatomy_protein_absent,expression absent,64523,2084,anatomy,heart left ventricle,UBERON,58254,105378952,gene/protein,KLF18,NCBI


In [4]:
# get all types to string
df = df.astype(str)
df.dtypes

relation            object
display_relation    object
x_index             object
x_id                object
x_type              object
x_name              object
x_source            object
y_index             object
y_id                object
y_type              object
y_name              object
y_source            object
dtype: object

# Get nodes

In [5]:
df_nodes = df[["x_index", "x_type", "x_name", "x_source"]]
df_nodes = df_nodes.rename(columns={"x_index": "y_index", "x_type": "y_type", "x_name": "y_name", "x_source": "y_source"})

# append the same columns with y_ prefix to df_nodes
df_nodes = pd.concat([df_nodes, df_nodes[["y_index", "y_type", "y_name", "y_source"]]], axis=0)

df_nodes = df_nodes.rename(columns={"y_index": "ID", "y_type": "type", "y_name": "name", "y_source": "source"})

print("Size of df_nodes before drop duplicates: ", df_nodes.shape[0])
# keep unique rows of df_nodes
df_nodes = df_nodes.drop_duplicates(ignore_index=False)
print("Size of df_nodes after drop duplicates: ", df_nodes.shape[0])

df_nodes

Size of df_nodes before drop duplicates:  16200996
Size of df_nodes after drop duplicates:  129375


,ID,type,name,source
0,0,gene/protein,PHYHIP,NCBI
1,1,gene/protein,GPANK1,NCBI
2,2,gene/protein,ZRSR2,NCBI
3,3,gene/protein,NRF1,NCBI
4,4,gene/protein,PI4KA,NCBI
...,...,...,...,...
6505728,129370,pathway,Ion transport by P-type ATPases,REACTOME
6505729,129371,pathway,Inhibition of voltage gated Ca2+ channels via...,REACTOME
6548634,129372,anatomy,anatomical entity,UBERON
6562425,129373,anatomy,multi-cellular organism,UBERON


In [6]:
import numpy as np
np.unique(df_nodes.type)

array(['anatomy', 'biological_process', 'cellular_component', 'disease',
       'drug', 'effect/phenotype', 'exposure', 'gene/protein',
       'molecular_function', 'pathway'], dtype=object)

In [7]:
df_nodes.to_csv("../prime/new/kg_nodes.csv", index=False)

# Get edges

In [8]:
df_edges = df[["relation", "display_relation"]]

# keep unique rows of df_nodes
df_edges = df_edges.drop_duplicates(ignore_index=False)

df_edges

,relation,display_relation
0,protein_protein,ppi
321075,drug_protein,carrier
321939,drug_protein,enzyme
327256,drug_protein,target
343636,drug_protein,transporter
346728,contraindication,contraindication
346730,indication,indication
347016,off-label use,off-label use
389359,drug_drug,synergistic interaction
3061987,phenotype_protein,associated with


In [10]:
len(set(df_edges.relation))

30

In [11]:
len(set(df_edges.display_relation))

18

So we want to use display relation as those are the true edges.

# Get node relation node list

In [12]:
df_relations = df[["x_index", "display_relation", "relation", "y_index"]]
df_relations["display_relation"] = df_relations["display_relation"].str.replace(" ", "_")
df_relations

/var/folders/7r/2xd4t0f94wd49mswwc461j940000gn/T/ipykernel_1785/1589442216.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relations["display_relation"] = df_relations["display_relation"].str.replace(" ", "_")


,x_index,display_relation,relation,y_index
0,0,ppi,protein_protein,8889
1,1,ppi,protein_protein,2798
2,2,ppi,protein_protein,5646
3,3,ppi,protein_protein,11592
4,4,ppi,protein_protein,2122
...,...,...,...,...
8100493,66747,expression_absent,anatomy_protein_absent,5259
8100494,63824,expression_absent,anatomy_protein_absent,58254
8100495,63826,expression_absent,anatomy_protein_absent,58254
8100496,64523,expression_absent,anatomy_protein_absent,58254


In [ ]:
def split_dataset(df, x):
    # Calculate the number of rows per subset
    total_rows = len(df)
    subset_size = total_rows // x

    # Create empty list to hold subsets
    subsets = []

    # Split the DataFrame into subsets
    for i in range(x):
        start_index = i * subset_size
        end_index = (i + 1) * subset_size if i < x - 1 else total_rows
        subset = df.iloc[start_index:end_index].reset_index(drop=True)
        subsets.append(subset)

    # Now subsets[i] contains the i-th subset
    # You can access each subset like subsets[0], subsets[1], etc.

    # Example usage: Print the first few rows of each subset
    for i, subset in enumerate(subsets):
        print(f"Subset {i+1} - Rows: {len(subset)}")
        print(subset.head())
        print()  # Separate subsets with a blank line
    return subsets

In [ ]:
df_datasets = split_dataset(df_relations, 10)

In [ ]:
for i, item in enumerate(df_datasets):
    item.to_csv(f"../prime/new/kg_relations_{i}.csv", index=False)

In [ ]:
df_relations.to_csv("../prime/new/kg_relations.csv", index=False)